<a href="https://colab.research.google.com/github/jmcaussade/Arreglo-Incongruencias-Juegos-Jardin/blob/main/ArregloAtributosJuegosMesaJardin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>IMPORTATIONS</h1>

In [2]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.1 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode

<h1>Connection to Google Drive</h1>


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
ruta = "/content/drive/MyDrive/Sodimac/ArregloIncongruenciasJuegosJardin/JuegosJardin.xlsx"
df = pd.read_excel(ruta)

<h1>MAIN</h1>

Visualization


In [ ]:
print(df)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

Manipulation

Transformation to long format of "Descripcion Local del Producto"

In [6]:
def PrintingDF(data_set):
  pd.set_option('display.width', 1000)
  print(data_set.to_string(index=False))


In [ ]:
print(df[["Descripción Local del Producto", "Descripción Web"]])

In [9]:
print(df.iloc[0]["Descripción Web"])


Set balcon nature turquesa Turquesa


In [ ]:
# restringienda dataframe a columnas importantes
selected_columns = ["SKU - Chile", 'Descripción Web', 'Tipo de mobiliario jardín', 'Modelo', 'Número de personas', 'Color']
clean_df = df.loc[:, selected_columns]

clean_df['Número de personas'] = clean_df['Número de personas'].replace([np.inf, -np.inf], np.nan).fillna(0)
clean_df['Número de personas'] = clean_df['Número de personas'].astype(int).astype(str)
PrintingDF(clean_df)

In [ ]:
# Add new columns for each selected column without checking if the values exist creating the "Name column + appears"

for name_column in selected_columns[2:]:
    clean_df.insert(clean_df.columns.get_loc(name_column) + 1, f"{name_column} appears", None)

# Print the DataFrame with the new columns
clean_df


In [9]:
#Checks if the value for each attributes appears
# Columnas con atributos obligatorios para buscar en desc web
selected_columns = ['Tipo de mobiliario jardín', 'Modelo', 'Número de personas', 'Color']

# Function to check if all values appear in the Descripción Web column in any case
def all_values_appear_in_web_description(row):
    description_web = unidecode(row["Descripción Web"]).lower()
    for col in selected_columns:
        value = unidecode(str(row[col])).lower()
        if col == "Tipo de mobiliario jardín":
            aux = value.split(" ")
            if len(aux) == 4:
                value = aux[-2] + " " + aux[-1]
            else:
                value = aux[-1]
        # Set the corresponding boolean column to True or False
        row[f"{col} appears"] = value in description_web
    # Set the "AllValuesAppear" column based on the condition
    row['AllValuesAppear'] = all(row[f"{col} appears"] for col in selected_columns)
    return row

# Apply the function to each row
clean_df = clean_df.apply(all_values_appear_in_web_description, axis=1)


In [ ]:
clean_df

In [ ]:
# highlight  all True rows
def highlight_rows(row):
    if row['AllValuesAppear']:
        return ['background-color: lightgreen'] * len(row)
    else:
        return [''] * len(row)

# Apply the custom styles to the DataFrame
styled_df = clean_df.style.apply(highlight_rows, axis=1)

# Display the styled DataFrame
styled_df

In [ ]:
# highlight False cells in red
def highlight_cells(value):
    if value is False:
        return 'background-color: red'
    elif value is True:
        return 'background-color: lightgreen'
    else:
        return ''

# Apply the custom styles to the DataFrame
styled_df = clean_df.style.applymap(highlight_cells, subset=pd.IndexSlice[:, [f"{col} appears" for col in selected_columns]])

# Display the styled DataFrame
styled_df

<h1>EXPORTING</h1>


In [12]:
# Assuming clean_df is your DataFrame
output_file_path = "/content/drive/MyDrive/Sodimac/ArregloIncongruenciasJuegosJardin/Results2.xlsx"

# Check if the file already exists
if os.path.exists(output_file_path):
    # Optionally, prompt the user or take appropriate action
    print(f"Warning: The file {output_file_path} already exists. It will be overwritten if the name is not changed.")
else:
  # Export the DataFrame to an Excel file
  clean_df.to_excel(output_file_path, index=False)
  print(f"DataFrame exported to {output_file_path}")



DataFrame exported to /content/drive/MyDrive/Sodimac/ArregloIncongruenciasJuegosJardin/Results2.xlsx
